# Lab 11-3

## Python Class

In [6]:
import numpy as np
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name, reuse=tf.AUTO_REUSE):
            self.X = tf.placeholder(tf.float32, shape=[None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, shape=[None, 10])

            self.keep_prob = tf.placeholder(tf.float32)
            learning_rate = 0.001

            # L1 ImgIn shape=(?, 28, 28, 1)
            W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
            #    Conv  shape=(?, 28, 28, 32)
            #    Pool  shape=(?, 14, 14, 32)
            L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
            L1 = tf.nn.relu(L1)
            L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            L1 = tf.nn.dropout(L1, keep_prob=self.keep_prob)

            # L2 ImgIn shape=(?, 14, 14, 32)
            W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
            #    Conv  shape=(?, 14, 14, 64)
            #    Pool  shape=(?, 7, 7, 64)
            L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
            L2 = tf.nn.relu(L2)
            L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            L2 = tf.nn.dropout(L2, keep_prob=self.keep_prob)

            # L3 ImgIn shape=(?, 7, 7, 64)
            W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
            #    Conv  shape=(?, 7, 7, 128)
            #    Pool  shape=(?, 4, 4, 128)
            L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
            L3 = tf.nn.relu(L3)
            L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            L3 = tf.nn.dropout(L3, keep_prob=self.keep_prob)
            L3 = tf.reshape(L3, [-1, 4 * 4 * 128])

            with tf.variable_scope("W4", reuse=tf.AUTO_REUSE) as scope:
                W4 = tf.get_variable("W4", shape=[4 * 4 * 128, 625], initializer=tf.contrib.layers.xavier_initializer())
            b4 = tf.Variable(tf.random_normal([625]))
            L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
            L4 = tf.nn.dropout(L4, keep_prob=self.keep_prob)

            with tf.variable_scope("W5", reuse=tf.AUTO_REUSE) as scope:
                W5 = tf.get_variable("W5", shape=[625, 10], initializer=tf.contrib.layers.xavier_initializer())
            b5 = tf.Variable(tf.random_normal([10]))
            
            self.hypothesis = tf.matmul(L4, W5) + b5
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.hypothesis, labels=self.Y))
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
            
            correct_prediction = tf.equal(tf.argmax(self.hypothesis, 1), tf.argmax(self.Y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, keep_prob=1.0):
        return self.sess.run(self.hypothesis, feed_dict={self.X: x_test, self.keep_prob: keep_prob})
    
    def get_accuracy(self, x_test, y_test, keep_prob=1.0):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.keep_prob: keep_prob})
    
    def train(self, x_data, y_data, keep_prob=0.7):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: x_data, self.Y: y_data, self.keep_prob: keep_prob})
            
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

tf.reset_default_graph()

sess = tf.Session()
m1 = Model(sess, "m1")

sess.run(tf.global_variables_initializer())

training_epochs = 15
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

print('Learning started. It takes sometime.')

for epoch in range(training_epochs):
    avg_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        c, _ = m1.train(batch_xs, batch_ys)
        
        avg_cost += c / total_batch
    
    print('Epoch: %04d' % (epoch + 1), 'cost = {:.9f}'.format(avg_cost))
    
print('Learning Finished!')
print('Accuracy:', m1.get_accuracy(mnist.test.images, mnist.test.labels))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.409242731
Epoch: 0002 cost = 0.085891434
Epoch: 0003 cost = 0.065335395
Epoch: 0004 cost = 0.053396829
Epoch: 0005 cost = 0.047547864
Epoch: 0006 cost = 0.044031710
Epoch: 0007 cost = 0.038944595
Epoch: 0008 cost = 0.036314502
Epoch: 0009 cost = 0.033527773
Epoch: 0010 cost = 0.030874100
Epoch: 0011 cost = 0.029271407
Epoch: 0012 cost = 0.027563748
Epoch: 0013 cost = 0.027710342
Epoch: 0014 cost = 0.026365754
Epoch: 0015 cost = 0.024335015
Learning Finished!
Accuracy: 0.9929


## tf.layers

```python
conv1 = tf.layers.conv2d(inputs=X_img, filter=32, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(input=conv1, pool_size=[2, 2], padding='SAME', strides=2)
dropout1 = tf.layers.dropout(inputs=pool1, rate=self.keep_prob, training=self.training)

...

flat = tf.reshape(dropout3, [-1, 4 * 4 * 128])

dense4 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=dense4, rate=self.keep_prob, training=self.training)

```

is equivalent for

```python
# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
#    Conv  shape=(?, 28, 28, 32)
#    Pool  shape=(?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob=self.keep_prob)

...

L3 = tf.reshape(L3, [-1, 4 * 4 * 128])

with tf.variable_scope("W4", reuse=tf.AUTO_REUSE) as scope:
    W4 = tf.get_variable("W4", shape=[4 * 4 * 128, 625], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=self.keep_prob)
```

## Ensemble

In [7]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

sess = tf.Session()

models = []
num_models = 7
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))
    
sess.run(tf.global_variables_initializer())

training_epochs = 15
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

print('Learning started. It takes sometime.')

for epoch in range(training_epochs):
    avg_cost = [0] * num_models
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        for index, model in enumerate(models):
            c, _ = model.train(batch_xs, batch_ys)

            avg_cost[index] += c / total_batch
    
    print('Epoch: %04d' % (epoch + 1), 'cost = ', avg_cost)
    
print('Learning Finished!')

test_size = len(mnist.test.labels)
predictions = np.zeros(test_size * 10).reshape(test_size, 10)

for index, model in enumerate(models):
    print(index, 'Accuracy:', model.get_accuracy(mnist.test.images, mnist.test.labels))
    p = model.predict(mnist.test.images)
    predictions += p
    
ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost =  [0.42265444201501956, 0.4283716221221471, 0.4787348500943998, 0.5029855599694624, 0.4261229236627168, 0.4012170760976995, 0.3487490368871525]
Epoch: 0002 cost =  [0.09800289445810705, 0.09407526589760723, 0.09156829074702483, 0.1023807594298639, 0.0907636711373926, 0.09897298017855408, 0.09718010613525453]
Epoch: 0003 cost =  [0.06814085557718172, 0.07048237936994567, 0.06564465607516476, 0.07296031905338164, 0.06492001147686755, 0.07106103611009375, 0.0695070692321117]
Epoch: 0004 cost =  [0.05872646928468544, 0.056847020964222846, 0.056605964043093045, 0.06271408763375473, 0.055970550934276095, 0.05875528884120287, 0.05852084967307747]
Epoch: 0005 cost =  [0.053660374216058036, 0.048915701731616114, 0.048096786608750115, 0.05328072556209